In [1]:

from binarybeech.binarybeech import *


In [2]:

df = pd.read_csv("data/titanic.csv")
df_titanic = df.sample(frac=0.75)
df_test = df.loc[df.index.difference(df_titanic),:]
    

In [3]:

gbt_titanic = GradientBoostedTree(df_titanic,"Survived",learning_rate=0.1)


In [4]:
gbt_titanic.X_names

['Pclass',
 'Name',
 'Sex',
 'Age',
 'Siblings/Spouses Aboard',
 'Parents/Children Aboard',
 'Fare']

In [5]:
it=gbt_titanic._initial_tree()
it.tree.show()


0.39097744360902253



In [6]:
it.tree.root

In [ ]:

gbt_titanic.create_trees(30)


5.49	 0.4086
5.51	 0.4398


In [ ]:

gbt_titanic.predict(df_titanic.iloc[0])

In [ ]:

gbt_titanic.validate()
